Machine Learning with Pytorch

PyTorch is an open source framework for AI research and commercial production in machine learning. It is used to build, train and optmize deep learning neural networks for applications such as image recognition, natural language processing, and speech recognition. It provides computation support for CPU, GPU, parallel and distributed training on multiple GPUs and multiple nodes. PyTorch is also flexible and easily extensible, with specific libraries and tools available for many different domains. All of the above have made PyTorch a leading framework in machine learning.

This lab shows you how easy is to get started with PyTorch and use it to build, train and evaluate a neural network.

Objectives

After completing this lab you will be able to:

    - Install necessary PyTorch libraries;
    - Use PyTorch to build, train and evaluate neural networks.
    - Save the trained model parameters and use them later for inferencing.


Setup

For this lab, we will be using the following libraries:

    - torch
    - TorchVision   

Importing Required Libraries

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

How does this lab work?

The lab uses MNIST datasets. The dataset has over 60,000 images of hand written digits. The data will be partitioned between training the AI model and testing the AI model after training.

The main steps in this project include:

1. Download the MNIST dataset and create a DataLoader for the dataset.
2. Define an AI model to recognize a hand written digit.
3. Train the defined AI model using training data from the MNIST dataset.
4. Test the trained AI model using testing data from the MNIST dataset.
5. Evaluate

Download Dataset and Create Data Loader

The images are 28x28 pixel images of digits 0 through 9.

In [9]:
# Download training data from the MNIST datasets.
training_data = datasets.MNIST(
    root="..\Pytorch\Datasets",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="..\Pytorch\Datasets",
    train=False, 
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders to iterate over data
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print("Training data size", len(train_dataloader) * batch_size)
print("Test data size:", len(test_dataloader) * batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Training data size 60032
Test data size: 10048
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:11: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:11: SyntaxWarning: invalid escape sequence '\P'
C:\Users\anthonny.paz\AppData\Local\Temp\ipykernel_9868\2042593826.py:3: SyntaxWarning: invalid escape sequence '\P'
  root="..\Pytorch\Datasets",
C:\Users\anthonny.paz\AppData\Local\Temp\ipykernel_9868\2042593826.py:11: SyntaxWarning: invalid escape sequence '\P'
  root="..\Pytorch\Datasets",


Define Model

We first determine the best device for perfoming training with cpu as the default device.

We then define the AI model as a neural network with 3 layers: an input layer, a hidden layer, and an output layer.
Between the layers, we use ReLU activation function.

Since the input images are 1x28x28 tensors, we need to flatten the input tensors into a 784 element tensor using the Flatten module before passing the input into our neural network.